In [38]:
def get_match(file_object, payload, label, pattern):
    case = payload.case
    hits = payload.hits
    length = 0
    for line in file_object:
        line = re.sub('\|',' ', line)
        length += len(line) # cummulative length of lines processed
        doc = nlp(line) # tokenize line

        matcher.add(label, None, pattern)
        matches = matcher(doc) 

        # match object returns a tuple with (id, startpos, endpos)
        for match_id, start, end in matches:
            span = doc[start: end]
            start, end = get_position(line, length, span.start_char, span.end_char) # get doc span
            #if {label: span.text, "span": (span.start_char, span.end_char)} not in hit: # line span
            if {label: span.text, "span": (start, end), "case": case} not in hits: 
                
                hits.append({label: span.text, "span": (start, end), "case": case})
                #print({label: span.text, "linespan": (span.start_char, span.end_char), "docspan": (start, end)})
            return hits

In [39]:
def get_position(line, length, start_char, end_char):
    '''
    get postion of hit in document
    '''
    #pos = [(token.text,token.idx,token.idx+len(token)) for token in doc]
    #for p in pos:
    #    print({"text":p[0],"start":p[1],"end":p[2]})
    
    l = len(line)
    if (int(l) > 0 and int(start_char) and int(end_char)):
        start = length - l + start_char
        end = length - l + end_char
        
        return start, end
    else:
        return None

In [41]:
def get_matches(payload):
    labels = list() # track current label
    #print(payload.patterns.items())
    
    for label, pattern in payload.patterns.items():
        #print(label)
        labels.append(label)
        file = io.open(payload.filename, 'r')
        if len(labels) > 1:
            #print('b', labels[len(labels)-2:len(labels)-1])
            matcher.remove(labels[len(labels)-2:len(labels)-1][0])
        matcher.add(label, None, pattern)
        #print(get_match(file, payload, label, pattern))
        h = get_match(file, payload, label, pattern)
    return h

if __name__ == '__main__':
    
    import spacy, io, re, os
    from spacy.matcher import Matcher 
    from spacy.attrs import *
    
    class FilePattern(object):
        def __init__(self,
                     filepath,
                     pattern):

            self = self
            self.filename = filename
            self.patterns = patterns
            self.case = os.path.basename(filename).split('-')[0]
            self.hits = list()

        @property
        def filePattern(self):
            return self
   
    # TODO: read files from directory using list or glob.glob
    filename = '/Users/gregsilverman/development/nlpie_dev/nlp/nlpie/projects/trauma/txt_files/pass_one/AC71231-v1.txt'
    
    # TODO: read patterns from file
    patterns = {
                "Age": [{"IS_DIGIT": True },{"LOWER": "years"}],
                "Sex": [{"LOWER":"male"}]
              }
    

    # instantiate 
    fp = FilePattern(filename,
                    patterns)
    
    #print(fp.filename,fp.patterns)
    
    nlp = spacy.load('en') 
    matcher = Matcher(nlp.vocab) 
    #hit = list()

    print(get_matches(fp))

[{'Age': '52 Years', 'span': (234, 242), 'case': 'AC71231'}, {'Sex': 'Male', 'span': (251, 255), 'case': 'AC71231'}]


In [1]:
import re
import spacy
nlp = spacy.load('en', disable=['parser', 'tagger', 'ner'])
doc = nlp("Car was going 44 mpHs.")

NUM_PATTERN = re.compile(r"(?i)\d+\s?mph")

for match in re.finditer(NUM_PATTERN, doc.text):
    start, end = match.span()
    print(f"The matched text: '{doc.text[start:end]}'", start, end)
    print(doc.char_span(start,end))
    span = doc.char_span(start, end)
    print(span)

The matched text: '44 mpH' 14 20
None
None
